# Classificação de Retinopatia Diabética

## Objetivo
Desenvolver um modelo de classificação binária para detectar a presença de retinopatia diabética em imagens de retina.

## Hipótese
"A diferença de tons gerais - do mais claro ao mais escuro - e presença de diferentes tons, indica a presença da retinopatia."

## Dataset
- **Saudáveis (No DR)**: 525 imagens
- **Doentes**: 1461 imagens (Mild: 370, Moderate: 599, Proliferate: 290, Severe: 202)

## 1. Instalação e Imports

In [ ]:
# Instalação das dependências
!pip install kagglehub tensorflow scikit-learn matplotlib seaborn opencv-python pillow tqdm -q

In [ ]:
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import cv2
from PIL import Image
from tqdm import tqdm
import warnings
warnings.filterwarnings('ignore')

# Machine Learning
from sklearn.model_selection import train_test_split, StratifiedKFold, cross_val_score
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score, roc_auc_score, roc_curve

# Deep Learning
import tensorflow as tf
from tensorflow.keras.models import Sequential, Model
from tensorflow.keras.layers import Dense, Dropout, Flatten, Conv2D, MaxPooling2D, BatchNormalization, GlobalAveragePooling2D
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau
from tensorflow.keras.applications import ResNet50, VGG16
from tensorflow.keras.optimizers import Adam

print(f"TensorFlow version: {tf.__version__}")
print(f"GPU disponível: {tf.config.list_physical_devices('GPU')}")

## 2. Download do Dataset

In [ ]:
import kagglehub

# Download do dataset
path = kagglehub.dataset_download("jockeroika/diabetic-retinopathy")
print("Caminho do dataset:", path)

In [ ]:
# Explorar estrutura do dataset
import os

def explore_directory(path, indent=0):
    """Explora e mostra a estrutura de diretórios"""
    items = os.listdir(path)
    for item in items:
        item_path = os.path.join(path, item)
        if os.path.isdir(item_path):
            files = len([f for f in os.listdir(item_path) if os.path.isfile(os.path.join(item_path, f))])
            print(" " * indent + f"📁 {item}/ ({files} arquivos)")
            if indent < 2:  # Limitar profundidade
                explore_directory(item_path, indent + 2)
        else:
            print(" " * indent + f"📄 {item}")

explore_directory(path)

## 3. Carregamento e Exploração dos Dados

In [ ]:
# Configurações
IMG_SIZE = 128  # Tamanho para redimensionar as imagens
RANDOM_STATE = 42

# Mapear classes para binário: 0 = Saudável, 1 = Doente
# Nomes das pastas no dataset: 'Healthy', 'Mild DR', 'Moderate DR', 'Proliferate DR', 'Severe DR'
CLASS_MAPPING = {
    'Healthy': 0,         # Saudável
    'Mild': 1,            # Doente (Mild DR)
    'Moderate': 1,        # Doente (Moderate DR)
    'Proliferate': 1,     # Doente (Proliferate DR)
    'Severe': 1           # Doente (Severe DR)
}

print("Mapeamento de classes:")
print("0 = Saudável (Healthy)")
print("1 = Doente (Mild DR, Moderate DR, Proliferate DR, Severe DR)")

In [ ]:
def load_images_from_folder(base_path, class_mapping, img_size=128):
    """
    Carrega imagens e converte para classificação binária.
    """
    images = []
    labels = []
    original_labels = []  # Para análise
    
    # Procurar por subpastas (train/test ou diretamente as classes)
    possible_paths = [
        base_path,
        os.path.join(base_path, 'train'),
        os.path.join(base_path, 'gaussian_filtered_images'),
        os.path.join(base_path, 'gaussian_filtered_images', 'gaussian_filtered_images')
    ]
    
    data_path = None
    for p in possible_paths:
        if os.path.exists(p):
            subdirs = [d for d in os.listdir(p) if os.path.isdir(os.path.join(p, d))]
            if any(key.lower() in d.lower() for d in subdirs for key in class_mapping.keys()):
                data_path = p
                break
    
    if data_path is None:
        print("Estrutura de diretórios encontrada:")
        explore_directory(base_path)
        raise ValueError("Não foi possível encontrar as pastas de classes. Verifique a estrutura.")
    
    print(f"Carregando imagens de: {data_path}")
    
    # Iterar sobre as classes
    for folder_name in os.listdir(data_path):
        folder_path = os.path.join(data_path, folder_name)
        
        if not os.path.isdir(folder_path):
            continue
        
        # Encontrar a classe correspondente
        binary_label = None
        for class_name, label in class_mapping.items():
            if class_name.lower() in folder_name.lower():
                binary_label = label
                break
        
        if binary_label is None:
            print(f"Pasta '{folder_name}' não mapeada, pulando...")
            continue
        
        print(f"Processando: {folder_name} -> {'Saudável' if binary_label == 0 else 'Doente'}")
        
        # Carregar imagens da pasta
        image_files = [f for f in os.listdir(folder_path) 
                      if f.lower().endswith(('.png', '.jpg', '.jpeg', '.tiff', '.bmp'))]
        
        for img_name in tqdm(image_files, desc=folder_name):
            img_path = os.path.join(folder_path, img_name)
            try:
                # Carregar e processar imagem
                img = cv2.imread(img_path)
                if img is None:
                    continue
                img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
                img = cv2.resize(img, (img_size, img_size))
                
                images.append(img)
                labels.append(binary_label)
                original_labels.append(folder_name)
            except Exception as e:
                print(f"Erro ao carregar {img_path}: {e}")
    
    return np.array(images), np.array(labels), original_labels

# Carregar dados
X, y, original_labels = load_images_from_folder(path, CLASS_MAPPING, IMG_SIZE)
print(f"\nTotal de imagens carregadas: {len(X)}")
print(f"Shape das imagens: {X.shape}")
print(f"Distribuição: Saudáveis={np.sum(y==0)}, Doentes={np.sum(y==1)}")

In [ ]:
# Visualizar distribuição das classes
fig, axes = plt.subplots(1, 2, figsize=(14, 5))

# Distribuição binária
class_counts = pd.Series(y).value_counts().sort_index()
colors = ['#2ecc71', '#e74c3c']
axes[0].bar(['Saudável (0)', 'Doente (1)'], class_counts.values, color=colors)
axes[0].set_title('Distribuição das Classes (Binária)')
axes[0].set_ylabel('Quantidade')
for i, v in enumerate(class_counts.values):
    axes[0].text(i, v + 10, str(v), ha='center', fontweight='bold')

# Distribuição original
orig_counts = pd.Series(original_labels).value_counts()
axes[1].bar(range(len(orig_counts)), orig_counts.values, color=plt.cm.viridis(np.linspace(0, 1, len(orig_counts))))
axes[1].set_xticks(range(len(orig_counts)))
axes[1].set_xticklabels(orig_counts.index, rotation=45, ha='right')
axes[1].set_title('Distribuição Original das Classes')
axes[1].set_ylabel('Quantidade')

plt.tight_layout()
plt.savefig('distribuicao_classes.png', dpi=150, bbox_inches='tight')
plt.show()

print(f"\nProporção: {np.sum(y==0)/len(y)*100:.1f}% Saudáveis, {np.sum(y==1)/len(y)*100:.1f}% Doentes")

In [ ]:
# Visualizar exemplos de imagens
fig, axes = plt.subplots(2, 5, figsize=(15, 6))

# Imagens saudáveis
healthy_idx = np.where(y == 0)[0]
for i, ax in enumerate(axes[0]):
    idx = healthy_idx[i]
    ax.imshow(X[idx])
    ax.set_title('Saudável', color='green')
    ax.axis('off')

# Imagens com retinopatia
sick_idx = np.where(y == 1)[0]
for i, ax in enumerate(axes[1]):
    idx = sick_idx[i]
    ax.imshow(X[idx])
    ax.set_title('Retinopatia', color='red')
    ax.axis('off')

plt.suptitle('Exemplos de Imagens de Retina', fontsize=14)
plt.tight_layout()
plt.savefig('exemplos_imagens.png', dpi=150, bbox_inches='tight')
plt.show()

## 4. Pré-processamento

### Análise de tons (baseado na hipótese)

In [ ]:
def extract_tone_features(images):
    """
    Extrai características de tons das imagens para validar a hipótese.
    - Média e desvio padrão dos canais RGB
    - Histograma de intensidades
    - Contraste
    """
    features = []
    
    for img in tqdm(images, desc="Extraindo features de tons"):
        # Converter para escala de cinza para análise de tons
        gray = cv2.cvtColor(img, cv2.COLOR_RGB2GRAY)
        
        # Features básicas
        feat = {
            'mean_intensity': np.mean(gray),
            'std_intensity': np.std(gray),
            'min_intensity': np.min(gray),
            'max_intensity': np.max(gray),
            'contrast': np.max(gray) - np.min(gray),
            'mean_r': np.mean(img[:,:,0]),
            'mean_g': np.mean(img[:,:,1]),
            'mean_b': np.mean(img[:,:,2]),
            'std_r': np.std(img[:,:,0]),
            'std_g': np.std(img[:,:,1]),
            'std_b': np.std(img[:,:,2]),
        }
        
        # Histograma em quartis
        hist, _ = np.histogram(gray, bins=4, range=(0, 256))
        hist = hist / hist.sum()  # Normalizar
        feat['hist_q1'] = hist[0]  # Tons escuros
        feat['hist_q2'] = hist[1]
        feat['hist_q3'] = hist[2]
        feat['hist_q4'] = hist[3]  # Tons claros
        
        features.append(feat)
    
    return pd.DataFrame(features)

# Extrair features de tons
tone_features = extract_tone_features(X)
tone_features['label'] = y
print(tone_features.head())

In [ ]:
# Visualizar diferenças de tons entre classes
fig, axes = plt.subplots(2, 3, figsize=(15, 10))

metrics = ['mean_intensity', 'std_intensity', 'contrast', 'mean_r', 'mean_g', 'mean_b']
titles = ['Intensidade Média', 'Desvio Padrão', 'Contraste', 'Média Canal R', 'Média Canal G', 'Média Canal B']

for ax, metric, title in zip(axes.flat, metrics, titles):
    healthy_data = tone_features[tone_features['label'] == 0][metric]
    sick_data = tone_features[tone_features['label'] == 1][metric]
    
    ax.boxplot([healthy_data, sick_data], labels=['Saudável', 'Doente'])
    ax.set_title(title)
    ax.set_ylabel('Valor')

plt.suptitle('Análise de Tons por Classe (Hipótese)', fontsize=14)
plt.tight_layout()
plt.savefig('analise_tons.png', dpi=150, bbox_inches='tight')
plt.show()

# Estatísticas descritivas por classe
print("\n=== Estatísticas por Classe ===")
print(tone_features.groupby('label')[metrics].mean())

In [ ]:
# Normalização das imagens para os modelos
X_normalized = X.astype('float32') / 255.0

# Divisão treino/teste (80/20) com estratificação
X_train, X_test, y_train, y_test = train_test_split(
    X_normalized, y, 
    test_size=0.2, 
    random_state=RANDOM_STATE, 
    stratify=y
)

print(f"Treino: {X_train.shape[0]} imagens")
print(f"Teste: {X_test.shape[0]} imagens")
print(f"\nDistribuição no treino: Saudáveis={np.sum(y_train==0)}, Doentes={np.sum(y_train==1)}")
print(f"Distribuição no teste: Saudáveis={np.sum(y_test==0)}, Doentes={np.sum(y_test==1)}")

In [ ]:
# Data Augmentation para o treinamento
datagen = ImageDataGenerator(
    rotation_range=15,
    width_shift_range=0.1,
    height_shift_range=0.1,
    horizontal_flip=True,
    vertical_flip=True,
    zoom_range=0.1,
    fill_mode='nearest'
)

datagen.fit(X_train)
print("Data augmentation configurado!")

## 5. Modelo Simples - MLP (Multi-Layer Perceptron)

Usaremos as features extraídas baseadas em tons + features flattened da imagem

In [ ]:
# Preparar dados para MLP (flatten + features de tons)
X_flat_train = X_train.reshape(X_train.shape[0], -1)
X_flat_test = X_test.reshape(X_test.shape[0], -1)

# Normalização com StandardScaler
scaler = StandardScaler()
X_flat_train_scaled = scaler.fit_transform(X_flat_train)
X_flat_test_scaled = scaler.transform(X_flat_test)

print(f"Shape para MLP: {X_flat_train_scaled.shape}")

In [ ]:
# Modelo MLP simples
def create_mlp_model(input_shape):
    model = Sequential([
        Dense(512, activation='relu', input_shape=(input_shape,)),
        BatchNormalization(),
        Dropout(0.5),
        
        Dense(256, activation='relu'),
        BatchNormalization(),
        Dropout(0.4),
        
        Dense(128, activation='relu'),
        BatchNormalization(),
        Dropout(0.3),
        
        Dense(1, activation='sigmoid')
    ])
    
    model.compile(
        optimizer=Adam(learning_rate=0.001),
        loss='binary_crossentropy',
        metrics=['accuracy']
    )
    
    return model

mlp_model = create_mlp_model(X_flat_train_scaled.shape[1])
mlp_model.summary()

In [ ]:
# Callbacks
early_stop = EarlyStopping(
    monitor='val_loss', 
    patience=10, 
    restore_best_weights=True
)

reduce_lr = ReduceLROnPlateau(
    monitor='val_loss', 
    factor=0.5, 
    patience=5, 
    min_lr=1e-6
)

# Calcular class weights para lidar com desbalanceamento
from sklearn.utils.class_weight import compute_class_weight
class_weights = compute_class_weight('balanced', classes=np.unique(y_train), y=y_train)
class_weight_dict = {i: w for i, w in enumerate(class_weights)}
print(f"Class weights: {class_weight_dict}")

In [ ]:
# Treinar MLP
print("=" * 50)
print("TREINANDO MODELO MLP (SIMPLES)")
print("=" * 50)

history_mlp = mlp_model.fit(
    X_flat_train_scaled, y_train,
    epochs=50,
    batch_size=32,
    validation_split=0.2,
    class_weight=class_weight_dict,
    callbacks=[early_stop, reduce_lr],
    verbose=1
)

In [ ]:
# Avaliar MLP
y_pred_mlp_prob = mlp_model.predict(X_flat_test_scaled)
y_pred_mlp = (y_pred_mlp_prob > 0.5).astype(int).flatten()

print("\n" + "=" * 50)
print("RESULTADOS - MODELO MLP (SIMPLES)")
print("=" * 50)
print("\nRelatório de Classificação:")
print(classification_report(y_test, y_pred_mlp, target_names=['Saudável', 'Doente']))

print(f"\nAcurácia: {accuracy_score(y_test, y_pred_mlp):.4f}")
print(f"AUC-ROC: {roc_auc_score(y_test, y_pred_mlp_prob):.4f}")

## 6. Modelo Avançado - CNN com Transfer Learning (ResNet50)

In [ ]:
# Modelo CNN avançado com Transfer Learning
def create_cnn_advanced(input_shape):
    # Usar ResNet50 pré-treinada
    base_model = ResNet50(
        weights='imagenet',
        include_top=False,
        input_shape=input_shape
    )
    
    # Congelar as camadas base inicialmente
    for layer in base_model.layers[:-20]:  # Descongelar últimas 20 camadas
        layer.trainable = False
    
    # Adicionar camadas de classificação
    x = base_model.output
    x = GlobalAveragePooling2D()(x)
    x = Dense(256, activation='relu')(x)
    x = BatchNormalization()(x)
    x = Dropout(0.5)(x)
    x = Dense(128, activation='relu')(x)
    x = BatchNormalization()(x)
    x = Dropout(0.3)(x)
    output = Dense(1, activation='sigmoid')(x)
    
    model = Model(inputs=base_model.input, outputs=output)
    
    model.compile(
        optimizer=Adam(learning_rate=0.0001),
        loss='binary_crossentropy',
        metrics=['accuracy']
    )
    
    return model

cnn_model = create_cnn_advanced((IMG_SIZE, IMG_SIZE, 3))
print(f"\nTotal de parâmetros: {cnn_model.count_params():,}")
print(f"Parâmetros treináveis: {sum([tf.keras.backend.count_params(w) for w in cnn_model.trainable_weights]):,}")

In [ ]:
# Treinar CNN avançada
print("\n" + "=" * 50)
print("TREINANDO MODELO CNN AVANÇADO (ResNet50)")
print("=" * 50)

early_stop_cnn = EarlyStopping(
    monitor='val_loss', 
    patience=15, 
    restore_best_weights=True
)

reduce_lr_cnn = ReduceLROnPlateau(
    monitor='val_loss', 
    factor=0.5, 
    patience=5, 
    min_lr=1e-7
)

history_cnn = cnn_model.fit(
    datagen.flow(X_train, y_train, batch_size=32),
    epochs=30,
    validation_data=(X_test, y_test),
    class_weight=class_weight_dict,
    callbacks=[early_stop_cnn, reduce_lr_cnn],
    verbose=1
)

In [ ]:
# Avaliar CNN
y_pred_cnn_prob = cnn_model.predict(X_test)
y_pred_cnn = (y_pred_cnn_prob > 0.5).astype(int).flatten()

print("\n" + "=" * 50)
print("RESULTADOS - MODELO CNN AVANÇADO (ResNet50)")
print("=" * 50)
print("\nRelatório de Classificação:")
print(classification_report(y_test, y_pred_cnn, target_names=['Saudável', 'Doente']))

print(f"\nAcurácia: {accuracy_score(y_test, y_pred_cnn):.4f}")
print(f"AUC-ROC: {roc_auc_score(y_test, y_pred_cnn_prob):.4f}")

## 7. Comparação dos Modelos

In [ ]:
# Função para plotar matriz de confusão
def plot_confusion_matrix(y_true, y_pred, title, ax):
    cm = confusion_matrix(y_true, y_pred)
    sns.heatmap(cm, annot=True, fmt='d', cmap='Blues', ax=ax,
                xticklabels=['Saudável', 'Doente'],
                yticklabels=['Saudável', 'Doente'])
    ax.set_title(title)
    ax.set_ylabel('Real')
    ax.set_xlabel('Predito')

# Comparar matrizes de confusão
fig, axes = plt.subplots(1, 2, figsize=(12, 5))

plot_confusion_matrix(y_test, y_pred_mlp, 'MLP (Modelo Simples)', axes[0])
plot_confusion_matrix(y_test, y_pred_cnn, 'CNN ResNet50 (Modelo Avançado)', axes[1])

plt.tight_layout()
plt.savefig('matrizes_confusao.png', dpi=150, bbox_inches='tight')
plt.show()

In [ ]:
# Curvas ROC
fig, ax = plt.subplots(figsize=(8, 6))

# MLP
fpr_mlp, tpr_mlp, _ = roc_curve(y_test, y_pred_mlp_prob)
auc_mlp = roc_auc_score(y_test, y_pred_mlp_prob)
ax.plot(fpr_mlp, tpr_mlp, label=f'MLP (AUC = {auc_mlp:.3f})', linewidth=2)

# CNN
fpr_cnn, tpr_cnn, _ = roc_curve(y_test, y_pred_cnn_prob)
auc_cnn = roc_auc_score(y_test, y_pred_cnn_prob)
ax.plot(fpr_cnn, tpr_cnn, label=f'CNN ResNet50 (AUC = {auc_cnn:.3f})', linewidth=2)

# Linha diagonal (classificador aleatório)
ax.plot([0, 1], [0, 1], 'k--', label='Aleatório (AUC = 0.5)')

ax.set_xlabel('Taxa de Falsos Positivos')
ax.set_ylabel('Taxa de Verdadeiros Positivos')
ax.set_title('Curvas ROC - Comparação dos Modelos')
ax.legend(loc='lower right')
ax.grid(True, alpha=0.3)

plt.savefig('curvas_roc.png', dpi=150, bbox_inches='tight')
plt.show()

In [ ]:
# Histórico de treinamento
fig, axes = plt.subplots(2, 2, figsize=(14, 10))

# MLP - Loss
axes[0, 0].plot(history_mlp.history['loss'], label='Treino')
axes[0, 0].plot(history_mlp.history['val_loss'], label='Validação')
axes[0, 0].set_title('MLP - Loss')
axes[0, 0].set_xlabel('Época')
axes[0, 0].set_ylabel('Loss')
axes[0, 0].legend()
axes[0, 0].grid(True, alpha=0.3)

# MLP - Accuracy
axes[0, 1].plot(history_mlp.history['accuracy'], label='Treino')
axes[0, 1].plot(history_mlp.history['val_accuracy'], label='Validação')
axes[0, 1].set_title('MLP - Acurácia')
axes[0, 1].set_xlabel('Época')
axes[0, 1].set_ylabel('Acurácia')
axes[0, 1].legend()
axes[0, 1].grid(True, alpha=0.3)

# CNN - Loss
axes[1, 0].plot(history_cnn.history['loss'], label='Treino')
axes[1, 0].plot(history_cnn.history['val_loss'], label='Validação')
axes[1, 0].set_title('CNN ResNet50 - Loss')
axes[1, 0].set_xlabel('Época')
axes[1, 0].set_ylabel('Loss')
axes[1, 0].legend()
axes[1, 0].grid(True, alpha=0.3)

# CNN - Accuracy
axes[1, 1].plot(history_cnn.history['accuracy'], label='Treino')
axes[1, 1].plot(history_cnn.history['val_accuracy'], label='Validação')
axes[1, 1].set_title('CNN ResNet50 - Acurácia')
axes[1, 1].set_xlabel('Época')
axes[1, 1].set_ylabel('Acurácia')
axes[1, 1].legend()
axes[1, 1].grid(True, alpha=0.3)

plt.tight_layout()
plt.savefig('historico_treinamento.png', dpi=150, bbox_inches='tight')
plt.show()

In [ ]:
# Tabela comparativa
from sklearn.metrics import precision_score, recall_score, f1_score

results = {
    'Métrica': ['Acurácia', 'Precisão', 'Recall', 'F1-Score', 'AUC-ROC'],
    'MLP (Simples)': [
        accuracy_score(y_test, y_pred_mlp),
        precision_score(y_test, y_pred_mlp),
        recall_score(y_test, y_pred_mlp),
        f1_score(y_test, y_pred_mlp),
        roc_auc_score(y_test, y_pred_mlp_prob)
    ],
    'CNN ResNet50 (Avançado)': [
        accuracy_score(y_test, y_pred_cnn),
        precision_score(y_test, y_pred_cnn),
        recall_score(y_test, y_pred_cnn),
        f1_score(y_test, y_pred_cnn),
        roc_auc_score(y_test, y_pred_cnn_prob)
    ]
}

df_results = pd.DataFrame(results)
df_results['MLP (Simples)'] = df_results['MLP (Simples)'].apply(lambda x: f'{x:.4f}')
df_results['CNN ResNet50 (Avançado)'] = df_results['CNN ResNet50 (Avançado)'].apply(lambda x: f'{x:.4f}')

print("\n" + "=" * 60)
print("COMPARAÇÃO FINAL DOS MODELOS")
print("=" * 60)
print(df_results.to_string(index=False))

## 8. Validação Cruzada

In [ ]:
# Validação Cruzada com K-Fold
from sklearn.model_selection import StratifiedKFold

n_folds = 5
skf = StratifiedKFold(n_splits=n_folds, shuffle=True, random_state=RANDOM_STATE)

print(f"\n{'='*60}")
print(f"VALIDAÇÃO CRUZADA ({n_folds}-FOLD)")
print(f"{'='*60}")

# Resultados para cada fold
mlp_cv_scores = []
cnn_cv_scores = []

for fold, (train_idx, val_idx) in enumerate(skf.split(X_normalized, y), 1):
    print(f"\n--- Fold {fold}/{n_folds} ---")
    
    # Dividir dados
    X_cv_train, X_cv_val = X_normalized[train_idx], X_normalized[val_idx]
    y_cv_train, y_cv_val = y[train_idx], y[val_idx]
    
    # MLP
    X_cv_train_flat = X_cv_train.reshape(X_cv_train.shape[0], -1)
    X_cv_val_flat = X_cv_val.reshape(X_cv_val.shape[0], -1)
    
    scaler_cv = StandardScaler()
    X_cv_train_scaled = scaler_cv.fit_transform(X_cv_train_flat)
    X_cv_val_scaled = scaler_cv.transform(X_cv_val_flat)
    
    mlp_cv = create_mlp_model(X_cv_train_scaled.shape[1])
    mlp_cv.fit(
        X_cv_train_scaled, y_cv_train,
        epochs=30,
        batch_size=32,
        validation_split=0.1,
        class_weight=class_weight_dict,
        callbacks=[EarlyStopping(patience=5, restore_best_weights=True)],
        verbose=0
    )
    
    mlp_cv_pred = (mlp_cv.predict(X_cv_val_scaled) > 0.5).astype(int).flatten()
    mlp_cv_scores.append(accuracy_score(y_cv_val, mlp_cv_pred))
    print(f"  MLP Acurácia: {mlp_cv_scores[-1]:.4f}")
    
    # CNN (simplificada para CV ser mais rápida)
    cnn_cv = create_cnn_advanced((IMG_SIZE, IMG_SIZE, 3))
    cnn_cv.fit(
        X_cv_train, y_cv_train,
        epochs=10,
        batch_size=32,
        validation_split=0.1,
        class_weight=class_weight_dict,
        callbacks=[EarlyStopping(patience=3, restore_best_weights=True)],
        verbose=0
    )
    
    cnn_cv_pred = (cnn_cv.predict(X_cv_val) > 0.5).astype(int).flatten()
    cnn_cv_scores.append(accuracy_score(y_cv_val, cnn_cv_pred))
    print(f"  CNN Acurácia: {cnn_cv_scores[-1]:.4f}")
    
    # Limpar memória
    del mlp_cv, cnn_cv
    tf.keras.backend.clear_session()

In [ ]:
# Resultados da Validação Cruzada
print(f"\n{'='*60}")
print("RESULTADOS DA VALIDAÇÃO CRUZADA")
print(f"{'='*60}")

print(f"\nMLP (Modelo Simples):")
print(f"  Scores por fold: {[f'{s:.4f}' for s in mlp_cv_scores]}")
print(f"  Média: {np.mean(mlp_cv_scores):.4f} (+/- {np.std(mlp_cv_scores)*2:.4f})")

print(f"\nCNN ResNet50 (Modelo Avançado):")
print(f"  Scores por fold: {[f'{s:.4f}' for s in cnn_cv_scores]}")
print(f"  Média: {np.mean(cnn_cv_scores):.4f} (+/- {np.std(cnn_cv_scores)*2:.4f})")

# Visualizar resultados CV
fig, ax = plt.subplots(figsize=(10, 6))

x = np.arange(n_folds)
width = 0.35

bars1 = ax.bar(x - width/2, mlp_cv_scores, width, label='MLP', color='steelblue')
bars2 = ax.bar(x + width/2, cnn_cv_scores, width, label='CNN ResNet50', color='coral')

ax.axhline(y=np.mean(mlp_cv_scores), color='steelblue', linestyle='--', alpha=0.7, label=f'MLP Média: {np.mean(mlp_cv_scores):.4f}')
ax.axhline(y=np.mean(cnn_cv_scores), color='coral', linestyle='--', alpha=0.7, label=f'CNN Média: {np.mean(cnn_cv_scores):.4f}')

ax.set_xlabel('Fold')
ax.set_ylabel('Acurácia')
ax.set_title('Validação Cruzada - Comparação dos Modelos')
ax.set_xticks(x)
ax.set_xticklabels([f'Fold {i+1}' for i in range(n_folds)])
ax.legend()
ax.set_ylim([0.5, 1.0])
ax.grid(True, alpha=0.3, axis='y')

plt.tight_layout()
plt.savefig('validacao_cruzada.png', dpi=150, bbox_inches='tight')
plt.show()

## 9. Conclusões e Relatório Final

In [ ]:
print("="*70)
print("RELATÓRIO FINAL - CLASSIFICAÇÃO DE RETINOPATIA DIABÉTICA")
print("="*70)

print("\n📊 DATASET:")
print(f"   - Total de imagens: {len(X)}")
print(f"   - Saudáveis: {np.sum(y==0)} ({np.sum(y==0)/len(y)*100:.1f}%)")
print(f"   - Doentes: {np.sum(y==1)} ({np.sum(y==1)/len(y)*100:.1f}%)")
print(f"   - Tamanho das imagens: {IMG_SIZE}x{IMG_SIZE} pixels")

print("\n🔬 HIPÓTESE:")
print("   'A diferença de tons gerais - do mais claro ao mais escuro - ")
print("    e presença de diferentes tons, indica a presença da retinopatia.'")

print("\n📈 RESULTADOS DOS MODELOS:")
print("\n   Modelo MLP (Simples):")
print(f"      - Acurácia no teste: {accuracy_score(y_test, y_pred_mlp):.4f}")
print(f"      - AUC-ROC: {roc_auc_score(y_test, y_pred_mlp_prob):.4f}")
print(f"      - Validação Cruzada: {np.mean(mlp_cv_scores):.4f} (+/- {np.std(mlp_cv_scores)*2:.4f})")

print("\n   Modelo CNN ResNet50 (Avançado):")
print(f"      - Acurácia no teste: {accuracy_score(y_test, y_pred_cnn):.4f}")
print(f"      - AUC-ROC: {roc_auc_score(y_test, y_pred_cnn_prob):.4f}")
print(f"      - Validação Cruzada: {np.mean(cnn_cv_scores):.4f} (+/- {np.std(cnn_cv_scores)*2:.4f})")

print("\n🏆 MELHOR MODELO:")
if np.mean(cnn_cv_scores) > np.mean(mlp_cv_scores):
    print("   CNN ResNet50 (Modelo Avançado)")
    print(f"   Melhoria sobre MLP: {(np.mean(cnn_cv_scores) - np.mean(mlp_cv_scores))*100:.2f}%")
else:
    print("   MLP (Modelo Simples)")
    print(f"   Melhoria sobre CNN: {(np.mean(mlp_cv_scores) - np.mean(cnn_cv_scores))*100:.2f}%")

print("\n📁 ARQUIVOS GERADOS:")
print("   - distribuicao_classes.png")
print("   - exemplos_imagens.png")
print("   - analise_tons.png")
print("   - matrizes_confusao.png")
print("   - curvas_roc.png")
print("   - historico_treinamento.png")
print("   - validacao_cruzada.png")

print("\n" + "="*70)

In [ ]:
# Salvar modelos
mlp_model.save('modelo_mlp_retinopatia.h5')
cnn_model.save('modelo_cnn_resnet50_retinopatia.h5')
print("Modelos salvos com sucesso!")